In [ ]:
import os
import numpy as np

# make sure the notebook is run from the root folder of maximal-empty-spheres
base_path = os.path.join(os.getcwd(), ".")

# Data

Download Armadillo, Koalla and Bunny meshes

In [ ]:
data_folder = os.path.join(base_path,"data")
if not os.path.exists(data_folder):
    os.mkdir(data_folder)
    
from utils.data import download_meshes
mesh_paths = download_meshes(data_folder)

print(f"\nAvailable Meshes: {list(mesh_paths.keys())}")

# GT Sampling

In [ ]:
import numpy as np
def sample_positions(n, random_sampling, sdf):
    if random_sampling:
        U = (np.random.rand(n*n*n, 3)-0.5)*2
    else:
        gx, gy, gz = np.meshgrid(np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1), np.linspace(-1.0, 1.0, n+1))
        U = np.vstack((gx.flatten(), gy.flatten(), gz.flatten())).T
    U_sdfvals = sdf(U)
    return U, U_sdfvals

import gpytoolbox as gpy

def GT_sdf(mesh_path):
    # Set up gt
    V_gt, F_gt = gpy.read_mesh(mesh_path)
    V_gt = gpy.normalize_points(V_gt)

    s = 0.9 # 0.75
    V_gt *= s/0.5

    # Create and abstract SDF function that is the only connection to the shape
    sdf = lambda x: gpy.signed_distance(x, V_gt, F_gt)[0]
    
    return (V_gt, F_gt), sdf

# Contouring

In [ ]:
from cgal.EmptySpheresReconstruction import MESReconstruction

# use the same screening weight for RFTA and MES
screening_weight = 1.

method_calls = {
    "RFTA" : lambda U,D,N=None : gpy.reach_for_the_arcs(U,D,screening_weight=screening_weight,parallel=True),
    "MES"  : lambda U,D,N=None : MESReconstruction(U,D,screening_weight=1.,cleanup=True),
    "MC"   : lambda U,D,N      : gpy.marching_cubes(D,U,*[N+1 for n in range(3)])
}

In [ ]:
out_folder = os.path.join(base_path,"output")
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

In [ ]:
mesh    = "armadillo"
Ns      = [10,] #,20,30,40]
methods = ["RFTA", "MES", "MC"]

# ---------------
for N in Ns:
    GT, sdf = GT_sdf(mesh_paths["armadillo"])
    U,D = sample_positions(N,False,sdf)
    
    for m in methods:
        if m in method_calls.keys():
            method_calls[m](U,D,N)

# Mesh Distances

In [ ]:
from utils.mesh_distances import ts_distances

In [ ]:
ts_distances(R_cgal, GT,2)

In [ ]:
def chamfer(v1,f1,v2,f2,n=1000000):
    P1 = gpy.random_points_on_mesh(v1,f1,n)
    P2 = gpy.random_points_on_mesh(v2,f2,n)
    d1 = gpy.squared_distance(P1,P2,use_aabb=True,use_cpp=True)[0]
    d2 = gpy.squared_distance(P2,P1,use_aabb=True,use_cpp=True)[0]
    return np.sqrt(np.mean(d1)) + np.sqrt(np.mean(d2))